In [2]:
import onnx
from onnx2pytorch import ConvertModel

onnx_model = onnx.load('Librispeech_Tensorflow_model.onnx')
pytorch_model = ConvertModel(onnx_model)

In [3]:
import pickle

with open('Librispeech_Tensorflow_X_train.pkl','rb') as f:
    X_train = pickle.load(f)
with open('Librispeech_Tensorflow_y_train.pkl','rb') as f:
    y_train = pickle.load(f)
with open('Librispeech_Tensorflow_X_val.pkl','rb') as f:
    X_val = pickle.load(f)
with open('Librispeech_Tensorflow_y_val.pkl','rb') as f:
    y_val = pickle.load(f)
with open('Librispeech_Tensorflow_X_test.pkl','rb') as f:
    X_test = pickle.load(f)
with open('Librispeech_Tensorflow_y_test.pkl','rb') as f:
    y_test = pickle.load(f)

In [4]:
pytorch_model.eval()

ConvertModel(
  (Gemm_sequential/dense/MatMul_Gemm__6:0): Linear(in_features=193, out_features=193, bias=True)
  (Relu_sequential/dense/Relu:0): ReLU(inplace=True)
  (Gemm_sequential/dense_1/MatMul_Gemm__7:0): Linear(in_features=128, out_features=193, bias=True)
  (Relu_sequential/dense_1/Relu:0): ReLU(inplace=True)
  (Gemm_sequential/dense_2/MatMul_Gemm__8:0): Linear(in_features=128, out_features=128, bias=True)
  (Relu_sequential/dense_2/Relu:0): ReLU(inplace=True)
  (Gemm_sequential/dense_3/MatMul_Gemm__9:0): Linear(in_features=251, out_features=128, bias=True)
  (Softmax_dense_3): Softmax(dim=-1)
)

In [20]:
import numpy as np
import torch

predicted = pytorch_model(torch.tensor(np.array([X_val[0]]), dtype=torch.float32)).argmax()

In [21]:
predicted

tensor(166)

In [23]:
y_val[0].argmax()

166

In [37]:
from torchinfo import summary

summary(pytorch_model, input_size=(1, 193))

Layer (type:depth-idx)                   Output Shape              Param #
ConvertModel                             [1, 251]                  --
├─Linear: 1-1                            [1, 193]                  37,442
├─ReLU: 1-2                              [1, 193]                  --
├─Linear: 1-3                            [1, 128]                  24,832
├─ReLU: 1-4                              [1, 128]                  --
├─Linear: 1-5                            [1, 128]                  16,512
├─ReLU: 1-6                              [1, 128]                  --
├─Linear: 1-7                            [1, 251]                  32,379
├─Softmax: 1-8                           [1, 251]                  --
Total params: 111,165
Trainable params: 111,165
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 0.11
Input size (MB): 0.00
Forward/backward pass size (MB): 0.01
Params size (MB): 0.44
Estimated Total Size (MB): 0.45

In [35]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, parameters, labels, device):
        self.parameters = parameters
        self.labels = labels
        self.device = device

    def __len__(self):
        return len(self.parameters)

    def __getitem__(self, idx):
        param = torch.tensor(self.parameters[idx], dtype=torch.float32).to(self.device)
        label = torch.tensor(self.labels[idx].argmax(), dtype=torch.long).to(self.device)
        return param, label

val_loader = DataLoader(CustomDataset(X_val, y_val, 'cpu'), batch_size=1)
pytorch_model.eval()
criterion = nn.CrossEntropyLoss()
val_loss_accumulator = 0.0
val_correct_accumulator = 0
val_total_samples = 0

with torch.no_grad():
    for data, targets in val_loader:
        outputs = pytorch_model(data)
        loss = criterion(outputs, targets)
        val_loss_accumulator += loss.item() * data.size(0)
        _, predicted = torch.max(outputs, 1)
        val_correct_accumulator += (predicted==targets).sum().item()
        val_total_samples += data.size(0)
    val_loss = val_loss_accumulator / val_total_samples
    val_accuracy = val_correct_accumulator / val_total_samples
print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {(val_accuracy * 100):.2f}")

Validation Loss: 4.5385, Validation Accuracy: 99.53
